In [ ]:
# !pip install qiskit

# from IPython.display import clear_output
# clear_output(wait=False)

In [1]:
import numpy as np
from qiskit import *
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp, Statevector
from tabulate import tabulate
from itertools import combinations
from qiskit.circuit.library import EfficientSU2
import math
import time
from qiskit.circuit.library import EfficientSU2
from qiskit.algorithms.time_evolvers.variational import ImaginaryMcLachlanPrinciple
from qiskit.algorithms import TimeEvolutionProblem
from qiskit.algorithms import VarQITE
from qiskit.primitives import Estimator
from qiskit.quantum_info import Statevector
from qiskit.algorithms import SciPyImaginaryEvolver
import pylab
from qiskit.algorithms.gradients import ReverseEstimatorGradient, ReverseQGT
from qiskit.circuit.library import RYGate
import matplotlib.pyplot as plt

import os
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
def tb0_JW(N,e,t) : 
    strings = []
    opt = SparsePauliOp.from_sparse_list([("I", [0], 0)], num_qubits=N)  
    for k in range(N) : 
        a0='I'*(N)
        a1 = 'I'*(k)+'Z' +'I'*(N-k-1)

        b0='I'*N
        b0_list = list(b0)
        b0_list[k] = 'X'
        b0_list[(k+1)%N] = 'X'
        new_b0 = ''.join(b0_list)

        b1='I'*N
        b1_list = list(b0)
        b1_list[k] = 'Y'
        b1_list[(k+1)%N] = 'Y'
        new_b1 = ''.join(b1_list)

        strings.append([a0,a1,new_b0,new_b1])
        val = 1
        if N==2 and k==1 : 
            val = 0
        opt += SparsePauliOp.from_list([(a0, 0.5*e[k]), (a1, -0.5*e[k]),(new_b0, 0.5*t*val),(new_b1, 0.5*t*val)])
    return opt  

def spinless_basis(N,r) : 
    basis_set = []
    lattice = list(range(N))
    places = list(combinations(lattice, r))
    for combination in places : 
        basis = [False] *N
        for index in combination : 
            basis[index] = True 
        basis_set.append(basis)
    return basis_set

def scs_param(n,l,var,param) : 

    circ = QuantumCircuit(n)
    circ.cx(-2,-1)
    circ.cry(param[var],-1,-2)
    var+=1
    circ.cx(-2,-1)

    for i in range(l-1) : 
        circ.cx(-3-i,-1)
        ccry = RYGate(param[var]).control(2,label=None)
        var+=1
        circ.append(ccry,[-1,-2-i,-3-i])
        circ.cx(-3-i,-1)
    return circ, var

def dicke_param(n,k) : 
    pairs = []
    for a in range(n,k,-1) : 
        pairs.append([a,k])
    for a in range(k,1,-1) : 
        pairs.append([a,a-1])

    num_angles = int(k*(n-k) + k*(k-1)/2)
    param = [Parameter(f"angle_{i+1}") for i in range(num_angles)]

    dk_circ = QuantumCircuit(n)
    dk_circ.x(range(-1,-k-1,-1))
    var=0
    for pair in pairs : 
        new_circ,new_var = scs_param(pair[0],pair[1],var,param)
        var = new_var
        dk_circ.append(new_circ, range(pair[0]))
    return dk_circ

In [3]:
backend = Aer.get_backend('statevector_simulator')

In [4]:
# folder_name = "Dicke plots"
# if not os.path.exists(folder_name):
#     os.makedirs(folder_name)

pdf_filename = os.path.join("Dicke_plots.pdf")
pdf_pages = PdfPages(pdf_filename)
results = []

for N in range(2,8) : 
    #Defining Hamiltonian
    e=[1]*N
    t=7
    H_op = tb0_JW(N,e,t)
    
    #Classically finding eigvectors and eigenvalues
    eig,vec = np.linalg.eig(H_op.to_matrix())
    new_vec = list(zip(*vec))

    eigstates = []
    for i,x in enumerate(eig) : 
        if np.round(x) == np.round(min(eig)) : 
            eigstates.append(new_vec[i])
            
    #Deciding ansatz       
    ansatz = dicke_param(N,N//2)
    
    init_param_values={}
    for i in range(len(ansatz.parameters)):
        init_param_values[ansatz.parameters[i]]=1
    
    exp_time = 5.0
    num_steps = 200
    
    aux_ops = [H_op]
    evolution_problem = TimeEvolutionProblem(H_op, exp_time, aux_operators=aux_ops)
    
    start_time = time.time()
    
    #var_principle = ImaginaryMcLachlanPrinciple()
    var_principle = ImaginaryMcLachlanPrinciple(qgt = ReverseQGT() , 
                                                gradient = ReverseEstimatorGradient())
    
    evolution_problem = TimeEvolutionProblem(H_op, exp_time, aux_operators=aux_ops)
    var_qite = VarQITE(ansatz, init_param_values, var_principle, Estimator(),
                      num_timesteps=num_steps)
    evolution_result_eff = var_qite.evolve(evolution_problem)
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    eff_circ = evolution_result_eff.evolved_state
    
    eff_job = execute(eff_circ, backend)
    eff_result = eff_job.result()
    eff_statevector = eff_result.get_statevector()
    
    sum_of_squares = (np.array(eff_statevector).conj() @ np.array(eff_statevector)).real
    norm_state = eff_statevector/np.sqrt(sum_of_squares)
    final_sv = [np.round(x,3) for x in np.asarray(norm_state)]
    
    overlap_list =[]
    for state in eigstates : 
        overlap_list.append(np.dot(state,np.conj(norm_state)))
    
    h_exp_val = np.array([ele[0][0] for ele in evolution_result_eff.observables])
    
    #Classical simulation
    init_state = Statevector(ansatz.assign_parameters(init_param_values))
    
    evolution_problem = TimeEvolutionProblem(H_op, exp_time, initial_state=init_state, 
                                             aux_operators=aux_ops)
    exact_evol = SciPyImaginaryEvolver(num_timesteps=num_steps)
    sol = exact_evol.evolve(evolution_problem)
    
    exact_h_exp_val = sol.observables[0][0].real
    #print("Exact lowest eigenvalue found : ",exact_h_exp_val[-1]) 
    
    times_eff = evolution_result_eff.times
    times_exact = sol.times
    
    pylab.plot(times_eff, h_exp_val, label= "VarQITE")
    pylab.plot(times_exact, exact_h_exp_val , label= "Exact",  linestyle='--')
    pylab.xlabel("Time")
    pylab.ylabel(r"$\langle H \rangle$ (energy)")
    pylab.legend(loc="upper right");
    
    
    pdf_pages.savefig()
    plt.close()
    
    results.append({'N' : N, 'expected eig' : np.round(min(eig),3).real, 'est_eig' : np.round(h_exp_val[-1],3), 
                    'time' : np.round(elapsed_time,3),'Eigstate accuracy' : np.round(overlap_list[0],3), 
                    'exact eig': np.round(exact_h_exp_val[-1],3) } )
    
pdf_pages.close()
df = pd.DataFrame(results)    

In [5]:
print(df)
df.to_csv('Dicke data.csv', index=False)

   N   expected eig  est_eig     time  Eigstate accuracy  exact eig
0  2  -6.000+0.000j   -6.000    5.722       1.000+0.000j     -6.000
1  3  -6.000+0.000j   -6.000   10.090      -0.497+0.000j     -6.000
2  4 -17.799+0.000j  -17.124   49.290      -0.988+0.000j    -17.799
3  5 -16.326+0.000j  -16.245   80.649      -0.077+0.000j    -16.326
4  6 -25.000+0.000j  -23.509  230.652       0.963+0.000j    -25.000
5  7 -25.342-0.000j  -24.376  379.062      -0.910+0.005j    -25.342
